In [116]:
import random
from pulp import *
from collections import deque
import time

# Ucitavanje grafa

In [117]:
def load_graph_from_mtx(filename):
    graph = {}
    with open(filename, 'r') as file:
        for i, line in enumerate(file):
            # preskocimo prva tri reda (jer su tu podaci o broju grana, cvorova...)
            if i < 3:
                continue
            
            nodes = line.strip().split()
            node1, node2 = int(nodes[0]), int(nodes[1])

            if node1 not in graph:
                graph[node1] = set()
            if node2 not in graph:
                graph[node2] = set()

            graph[node1].add(node2)
            graph[node2].add(node1)

    return graph


# Generisanje random burning sekvenci

In [118]:

def generate_random_burning_sequence(graph):
    
    S=[] # sekvenca zapaljenih cvoreva
    B= set() # skup zapaljenih cvoreva
    NB= set(graph.keys()) # skup nezapaljenih cvoreva, inicajlno su to svi cvorevi grafa
    
    # ako imamo vise kandidata, biramo onog sa najvisim betweenness
    
    
    current_node = random.choice(list(NB))
    
   
    S.append(current_node)
    B.add(current_node)
    NB.remove(current_node)
    
    while len(B)<len(graph):
       
        # korak 2: zapaliti sve susjede (koji nisu zapaljeni) od cvorova u skupu zapaljenih, dodajemo ih u B, brisemo iz NB
        
        # Provjera: ako su svi cvorovi u NB zapravo cvorovi koji ce biti zapaljeni u narednom koraku (jer su direktni susjedi
        # zapaljenom cvoru nekom), onda cemo izabrati neki random cvor iz NB cisto kao simbol jos jednog koraka
        
        FB = []
        for b in B:
            neighbors_b1 = graph[b]
            for neighbor in neighbors_b1:
                if neighbor in NB:
                    if neighbor not in FB:
                        FB.append(neighbor)
             
        if set(FB) == NB:
            random_node = random.choice(list(NB))
            S.append(random_node)
          
        newlyburned = []
        for node in B:
            neighbors = graph[node]  # svi susjedi trenutnog zapaljenog cvora
            for neighbor in neighbors:
                if neighbor in NB:  # ako susjed nije zapaljen
                    newlyburned.append(neighbor)  # dodajemo ga u zapaljene
                    NB.remove(neighbor)  
        
        for new in newlyburned:
            B.add(new) #dodajemo sve susjede u zapaljene
        
        # pravimo listu onih cvorova za koje znamo da ce se u iducem koraku zapaliti. Bolje da izaberemo sljedeci direktni
        # cvor za koji znamo da nece biti zapaljen u narednom koraku
        
        FB2 = []
        for b in B:
            neighbors_b = graph[b]
            for neighbor in neighbors_b:
                if neighbor in NB:
                    if neighbor not in FB2:
                        FB2.append(neighbor)
       
        # korak 3: pronaci iduci cvor koji ce biti direktno zapaljen, dodati ga u S, B i izbrisati iz NB
        
        max_burned = -1
        best_node = None
        candidates_for_best = []
        
        if NB:
            for node in NB:
                if node not in FB2: 
                    candidates_for_best.append(node)
            if candidates_for_best:
                best_node = random.choice(candidates_for_best)
            else:
                best_node = None        
                
            if best_node is not None:
                S.append(best_node)
                B.add(best_node)
                NB.remove(best_node)
            else:
             # Ako nema nijednog najboljeg cvora znaci da su svi preostali cvorovi povezani i mogu biti zapaljeni
             # Dodajemo slucajni cvor iz preostalih nezapaljenih čvorova
                random_node = random.choice(list(NB))
                S.append(random_node)
                B.add(random_node)
                NB.remove(random_node)
           
            
        else:
            break
           
    return S
        
        

def generate_multiple_random_burning_sequences(graph, num_sequences):
    
    sequences = []
    for _ in range(num_sequences):
        sequence = generate_random_burning_sequence(graph)
        if sequence is not None:  
            sequences.append(sequence)
    return sequences


filename = "grafovi/manji/karate.mtx"
graph = load_graph_from_mtx(filename)


#max_burn_nodes = len(graph) // 2
num_sequences = 30  

burning_sequences = generate_multiple_random_burning_sequences(graph, num_sequences)

# Ispis rezultata
print(f"Generisane burning sekvence:")
for seq in burning_sequences:
    print(seq,"len:", len(seq))


Generisane burning sekvence:
[31, 26, 17, 13] len: 4
[31, 11, 13, 17] len: 4
[18, 26, 19, 21, 15] len: 5
[6, 23, 21, 29] len: 4
[17, 21, 25, 22] len: 4
[5, 25, 27, 16, 15] len: 5
[13, 16, 19, 23] len: 4
[17, 3, 27, 26] len: 4
[10, 17, 25, 22] len: 4
[10, 17, 18, 26] len: 4
[30, 4, 17, 12] len: 4
[14, 17, 25, 26] len: 4
[7, 30, 25, 23] len: 4
[10, 7, 12, 25] len: 4
[1, 21, 24, 30] len: 4
[26, 12, 17, 20] len: 4
[2, 16, 21, 26] len: 4
[4, 15, 27, 17] len: 4
[23, 12, 17, 2] len: 4
[22, 34, 26, 17] len: 4
[34, 22, 17, 8] len: 4
[17, 3, 26, 27] len: 4
[24, 17, 12, 18, 22] len: 5
[24, 18, 17, 5] len: 4
[26, 16, 17, 31] len: 4
[6, 25, 19, 16, 23] len: 5
[5, 33, 26, 29] len: 4
[14, 25, 17] len: 3
[12, 29, 25, 24] len: 4
[15, 2, 17, 5] len: 4


In [119]:

def generate_distance_matrix(G):
    
    n = len(G)
    nodes = list(G.keys())  
    index_map = {v: i for i, v in enumerate(nodes)}  
    
    d = [[float('inf')] * n for _ in range(n)]
    
    for start in nodes:
        start_index = index_map[start]
        d[start_index][start_index] = 0  
        queue = deque([start])
        while queue:
            current = queue.popleft()
            current_index = index_map[current]
            for neighbor in G[current]:
                neighbor_index = index_map[neighbor]
                if d[start_index][neighbor_index] == float('inf'): 
                    d[start_index][neighbor_index] = d[start_index][current_index] + 1
                    queue.append(neighbor)
    
    return d

def ILP_Pulp(L, k, G, burning_sequences=None):
    
    n = len(G)  
    d = generate_distance_matrix(G) 
    
    
    m = LpProblem("Graph_burning", LpMinimize)

    # binarne promjenljive
    b = [LpVariable(f"b_{j}", cat='Binary') for j in range(n)]
    x = [[LpVariable(f"x_{r}_{i}", cat='Binary') for i in range(n)] for r in range(k)]

    # funkcija cilja (11)
    m += lpSum(x[r][i] for r in range(k) for i in range(n))

    # ogranicenje (12)
    for i in range(1, k):
        m += lpSum(x[i][j] for j in range(n)) <= lpSum(x[i-1][j] for j in range(n))

    # ogranicenje (13)
    for i in range(k):
        m += lpSum(x[i][j] for j in range(n)) <= 1

    # ogranicenje (14)
    for j in range(n):
        m += b[j] <= lpSum(x[i][k] for i in range(k) for k in range(n) if d[k][j] <= i)

    # ogranicenje (15)
    m += lpSum(b[j] for j in range(n)) == n

    # ogranicenje (17)
    m += lpSum(x[0][j] for j in range(n)) == 1


    # provjera svake sekvence iz liste `burning_sequences`
    best_sequence = None
    best_obj_value = float('inf')
    results = []
    if burning_sequences:
        for seq in burning_sequences:
            m_copy = m.deepcopy() 

            # ogranicenja za trenutnu sekvencu
            for r, node in enumerate(seq):
                for i in range(n):
                    if i == node:
                        m_copy += x[r][i] == 1
                    else:
                        m_copy += x[r][i] == 0
            
           
            m_copy.solve()
            status = LpStatus[m_copy.status]
            obj_value = value(m_copy.objective)
            
            
            results.append({
                "sequence": seq,
                "status": status,
                "objective_value": obj_value
            })
            
            # azuriraj najbolju sekvencu
            if status == "Optimal" and obj_value < best_obj_value:
                best_obj_value = obj_value
                best_sequence = seq
    
    return results, best_sequence, best_obj_value

startTime = time.time()

k=10

results, best_sequence, best_obj_value = ILP_Pulp(LpMinimize, k, graph, burning_sequences=burning_sequences)


# Ispis najbolje sekvence
print("\nBest Sequence:")
print(f"Sequence: {best_sequence}")
print(f"Objective Value: {best_obj_value}")

print("--- %s sekundi ---" % (time.time() - startTime))



Best Sequence:
Sequence: [31, 26, 17, 13]
Objective Value: 4.0
--- 2.74027156829834 sekundi ---
